In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [3]:
import pandas as pd
import re,string

In [3]:
# Load the tokenizer and the GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [4]:
# Set the maximum length of the generated text
# # Sample example
# max_length = 50

# # Generate text
# prompt = "Once upon a time"
# input_ids = tokenizer.encode(prompt, return_tensors='pt')
# output = model.generate(input_ids, max_length=max_length, do_sample=True)

# # Decode the generated text
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# print(generated_text)

In [4]:
df = pd.read_csv("rsp_story_data.csv")
df_trimmed = df.drop(columns=['ExternalId','ReleaseUId','Unnamed: 0'])
df_trimmed = df_trimmed.dropna(axis=0)
df_trimmed['StoryTitleDescription'] = df_trimmed['Title'] + " " + df_trimmed['Description']
df_trimmed['Acc'] = df_trimmed['AcceptanceCriteria']
df_trimmed = df_trimmed.drop(columns=['Title','Description'],axis=1)
df_trimmed = df_trimmed.drop(columns=['AcceptanceCriteria'])

In [5]:
df.shape

(12433, 6)

In [6]:
df_trimmed = df.drop(columns=['ExternalId','ReleaseUId','Unnamed: 0'])

In [7]:
df_trimmed.columns

Index(['Title', 'Description', 'AcceptanceCriteria'], dtype='object')

In [8]:
df_trimmed.isna().sum()

Title                    0
Description            877
AcceptanceCriteria    1150
dtype: int64

In [9]:
df_trimmed = df_trimmed.dropna(axis=0)

In [10]:
df_trimmed.isna().sum()

Title                 0
Description           0
AcceptanceCriteria    0
dtype: int64

In [11]:
df_trimmed.shape

(10600, 3)

In [12]:
df_trimmed['StoryTitleDescription'] = df_trimmed['Title'] + " " + df_trimmed['Description']
df_trimmed['Acc'] = df_trimmed['AcceptanceCriteria']
df_trimmed = df_trimmed.drop(columns=['Title','Description'],axis=1)
df_trimmed = df_trimmed.drop(columns=['AcceptanceCriteria'])

In [13]:
df_trimmed

,StoryTitleDescription,Acc
1,As RSP[UI] user when I click on the bubble in ...,The bubble chart is clickable Clicking the bub...
2,As a R360 phoenix user Audit Fields present in...,"The audit fields CreatedBy, CreatedAtSourceBy..."
4,As RSP[UI] user I should get the summary of de...,Widget should show the right value of Objects\...
5,As RSP[UI] user I should click on Hotspots in ...,Top pane should have the Overall Technical\nHo...
6,As a RSP[UI] user I must be able to view detai...,Help section in RSP should have the fo...
...,...,...
12428,Data inactivation for past releases in splashb...,The data on the backend should be cleared for ...
12429,As a phoenix user Audit Fields present in Phoe...,"The existing data to be encrypted . also, the ..."
12430,As a phoenix user Audit Fields present in Phoe...,"The existing data to be encrypted . also, the ..."
12431,VA | Confidential popup vulnerability fix Whil...,Cookie creation details to be modified to incl...


In [16]:
df_trimmed.iloc[4].StoryTitleDescription

'As a RSP[UI] user I must be able to view details in Help icon for RSP   Help section in RSP should have the following details: User Guide FAQs    '

In [15]:
df_trimmed['StoryTitleDescription']

1        As RSP[UI] user when I click on the bubble in ...
2        As a R360 phoenix user Audit Fields present in...
4        As RSP[UI] user I should get the summary of de...
5        As RSP[UI] user I should click on Hotspots in ...
6        As a RSP[UI] user I must be able to view detai...
                               ...                        
12428    Data inactivation for past releases in splashb...
12429    As a phoenix user Audit Fields present in Phoe...
12430    As a phoenix user Audit Fields present in Phoe...
12431    VA | Confidential popup vulnerability fix Whil...
12432    As a phoenix user Audit Fields present in Phoe...
Name: StoryTitleDescription, Length: 10600, dtype: object

In [28]:
def remove_punctuation(input):
    #puctuation_pattern = re.compile(r'[%s]'%string.punctuation
    puctuation_pattern=re.compile(r'[!"#$%&\'()*+-/:;<=>?@[\]^_`{|}~]', re.UNICODE)
    input = puctuation_pattern.sub(' ', input)
    input = input.replace('  ',' ')
    input = input.replace("\n",' ')
    return input
df_trimmed['StoryTitleDescription'] = df_trimmed['StoryTitleDescription'].apply(lambda x : remove_punctuation(x))
df_trimmed['Acc'] = df_trimmed['Acc'].apply(lambda x : remove_punctuation(x))

In [29]:
df_trimmed['StoryTitleDescription'] = df_trimmed['StoryTitleDescription'].apply(lambda x : remove_punctuation(x))
df_trimmed['Acc'] = df_trimmed['Acc'].apply(lambda x : remove_punctuation(x))

In [30]:
df_trimmed

,StoryTitleDescription,Acc
1,As RSP UI user when I click on the bubble in H...,The bubble chart is clickable Clicking the bub...
2,As a R360 phoenix user Audit Fields present in...,The audit fields CreatedBy CreatedAtSourceByU...
4,As RSP UI user I should get the summary of def...,Widget should show the right value of Objects ...
5,As RSP UI user I should click on Hotspots in T...,Top pane should have the Overall Technical Hot...
6,As a RSP UI user I must be able to view detail...,Help section in RSP should have the follow...
...,...,...
12428,Data inactivation for past releases in splashb...,The data on the backend should be cleared for ...
12429,As a phoenix user Audit Fields present in Phoe...,The existing data to be encrypted also the ne...
12430,As a phoenix user Audit Fields present in Phoe...,The existing data to be encrypted also the ne...
12431,VA Confidential popup vulnerability fix While...,Cookie creation details to be modified to incl...


# FINE TUNING  BART 

In [67]:
from transformers import BartForConditionalGeneration,BartTokenizer

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')


Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [57]:
stories = df_trimmed.StoryTitleDescription.tolist()
acc_criteria = df_trimmed.Acc.tolist()

In [58]:
encoded_inputs = tokenizer(stories, acc_criteria, padding=True, truncation=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [62]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
class CustomDataset(Dataset):
    def __init__(self, encoded_inputs):
        self.encoded_inputs = encoded_inputs

    def __len__(self):
        return len(self.encoded_inputs['input_ids'])

    def __getitem__(self, idx):
        return {'user_story_input': {'input_ids': torch.tensor(self.encoded_inputs['input_ids'][idx]),
                                     'attention_mask': torch.tensor(self.encoded_inputs['attention_mask'][idx])},
                'ac_input': {'input_ids': torch.tensor(self.encoded_inputs['decoder_input_ids'][idx]),
                             'attention_mask': torch.tensor(self.encoded_inputs['decoder_attention_mask'][idx])},
                'labels': torch.tensor(self.encoded_inputs['decoder_input_ids'][idx])}

train_dataset = CustomDataset(encoded_inputs)

In [78]:
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments
#from transformers.modeling_bart 
from transformers.models.bart.modeling_bart import shift_tokens_right

In [82]:
def convert_to_features(stories,acc):
    input_encodings = tokenizer.batch_encode_plus(stories, pad_to_max_length=True, max_length=1024, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(acc, pad_to_max_length=True, max_length=1024, truncation=True)
    
    labels = target_encodings['input_ids']
    decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id,decoder_start_token_id=1)
    labels[labels[:, :] == model.config.pad_token_id] = -100
    
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': decoder_input_ids,
        'labels': labels,
    }

    return encodings

In [83]:
dataset=convert_to_features(stories,acc_criteria)

AttributeError: 'list' object has no attribute 'new_zeros'

In [73]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# Load pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Define the input/output format for the model
class UserStoryACInput(torch.nn.Module):
    def __init__(self):
        super(UserStoryACInput, self).__init__()
        self.encoder = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None):
        encoder_outputs = self.encoder(input_ids, attention_mask=attention_mask)
        decoder_outputs = self.encoder(decoder_input_ids, attention_mask=decoder_attention_mask, encoder_outputs=encoder_outputs)
        return decoder_outputs.logits
    
# Prepare the training dataset
#train_dataset = [...]  # You need to define this based on your use case
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)




loading configuration file config.json from cache at C:\Users\kammari.vara.prasad/.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\config.json
Model config BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  }

In [75]:
# Tokenize input and target texts
input_encodings = tokenizer(stories, truncation=True, padding=True)
target_encodings = tokenizer(acc_criteria, truncation=True, padding=True)

# Convert tokenized input and target texts to tensors
input_ids = torch.tensor(input_encodings['input_ids'])
attention_mask = torch.tensor(input_encodings['attention_mask'])
target_ids = torch.tensor(target_encodings['input_ids'])
target_attention_mask = torch.tensor(target_encodings['attention_mask'])

# Create dataset|
train_dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, target_ids, target_attention_mask)

In [76]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=1000,
    seed=42,
    evaluation_strategy='steps',
    eval_steps=1000,
    warmup_steps=1000,
    prediction_loss_only=True,
    disable_tqdm=False,  # enable progress bar
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)

# Define the trainer
trainer = Trainer(
    model=UserStoryACInput(),
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\kammari.vara.prasad/.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\config.json
Model config BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "enc

AttributeError: 'tuple' object has no attribute 'keys'

In [31]:
with open("rsp_story1.txt", 'w', encoding = 'utf-8') as f:
    for index,row in df_trimmed.iterrows():
        text = row['StoryTitleDescription'] + " <SEP> " + row['Acc']
        f.write(text + '\n')
    

In [32]:
from transformers import GPT2Tokenizer, LineByLineTextDataset

# Load the data file
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='rsp_story1.txt',
    block_size=128
)


C:\Users\kammari.vara.prasad\Anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [33]:
from sklearn.model_selection import train_test_split



In [34]:
# Split the dataset into train, validation, and test sets
from sklearn.model_selection import train_test_split
train_val, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

In [49]:
train[0]

{'input_ids': tensor([45147, 27710,   284, 15000,  3497, 30116, 11792, 33129, 41571,  7824,
           351, 27564,  2043,  4235,  2427,   286, 14392,  4235,   284, 20580,
         15193,  1675,  3440,   262,  8354, 31870,   287,   262, 12454,  5443,
           775,   423,  2904,  9177,   257,  1487,   284,  8354, 31870,  7824,
           810,   356,  3017,   257,   564,   250, 32374,   447,   251,  4235,
           326,  5860,  1342,  2033,   286,  1366,   220,  9643,  1541, 11521,
           428,   284,   262, 12454,   355,   636,   286,   371,    19,   352,
           290,   994,   389,   262,  3307,  2174,   284,  3494,   329,   428,
          3586,  7824, 10289,   220,  3740,   220,  1957,  4774, 40384,  1485,
          4755,   410,    16,  3497, 30116, 11792, 33129, 42316,    82,  5456,
            52,  7390,  3571,  3064,   830, 17643, 17643, 17643, 27551,  2388,
          7585, 42316,    52,  7390,  9242,  2291, 20988,    39,   959,  9282,
          2081,  3053,  1332,  1270,   

In [38]:
print(f"Length of training data: {len(train)}")
print(f"Length of validation data: {len(val)}")
print(f"Length of testing data: {len(test)}")

Length of training data: 6784
Length of validation data: 1696
Length of testing data: 2120


In [46]:
from transformers import DataCollatorForLanguageModeling,TrainingArguments,Trainer
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # set mlm to True if using a masked language model
)


In [47]:
tokenizer.pad_token = tokenizer.eos_token

In [45]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=1000,
    learning_rate=5e-5,
    warmup_steps=1000,
    prediction_loss_only=True,
    disable_tqdm=False,  # enable progress bar
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    data_collator=data_collator
)
trainer.train()

In [ ]:
model.train()

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [ ]:
input_output_pairs =[]

for index,row in df_trimmed.iterrows():
    temp={}
    temp['input']=row['StoryTitleDescription']
    temp['output']=row['Acc']
    input_output_pairs.append(temp)

In [ ]:
from torch.utils.data import DataLoader

# Create a DataLoader object
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
# Tokenize the input-output pairs
tokenized_pairs = [{'input': tokenizer.encode(pair['input']), 'output': tokenizer.encode(pair['output'])} for pair in input_output_pairs]

In [ ]:
df1=pd.DataFrame(input_output_pairs)

In [ ]:
df1.columns

In [ ]:
df1.isna().sum()

In [ ]:

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=1000,
    learning_rate=5e-5,
    warmup_steps=1000,
    prediction_loss_only=True,
    disable_tqdm=False,  # enable progress bar
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)


In [84]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned GPT-2 model
model = GPT2LMHeadModel.from_pretrained('./results/checkpoint-3000')

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Define a function to generate acceptance criteria
def generate_acceptance_criteria(user_story_title, user_story_description):
    # Encode the input prompt as input_ids
    input_ids = tokenizer.encode(f"User Story Title: {user_story_title}\nUser Story Description: {user_story_description}\n\nAcceptance Criteria:")

    # Generate the output text using the model and tokenizer
    output_text = model.generate(input_ids=input_ids,
                                 max_length=1024,
                                 temperature=0.7,
                                 do_sample=True,
                                 top_k=50,
                                 top_p=0.95,
                                 num_return_sequences=1)

    # Decode the generated output text
    generated_text = tokenizer.decode(output_text[0], skip_special_tokens=True)

    return generated_text


loading configuration file ./results/checkpoint-3000\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

loa

In [85]:
# Generate acceptance criteria for a new user story
user_story_title = "As RSP user , I want to see the dependent stories for current release stories"
user_story_description = "Each story should have the mapping to its dependent stories with dotted representation if it has depedency with the current release story"


In [86]:
input_text = user_story_title + " " + user_story_description

In [87]:
input_text
tokenizer.pad_token = tokenizer.eos_token

In [88]:
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")

In [94]:
def generate_acceptance_criteria(title, description):
    input_text = f"{title}\n\n{description}\n\nAcceptance Criteria:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output_ids = model.generate(input_ids, max_length=400, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text.strip()

In [95]:
generate_acceptance_criteria(user_story_title,user_story_description)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'As RSP user, I want to see the dependent stories for current release stories\n\nEach story should have the mapping to its dependent stories with dotted representation if it has depedency with the current release story\n\nAcceptance Criteria: Stories should be categorized based on acceptance criteria   <SEP> Given I am in RP app When I navigate to Dependency Builder page Then I must be able to select the release from the dropdown and view the Dependencies  The following calculation should take place  Total number of stories in the selected release category  Stories with Depends on average of 4 Acceptance criteria Total Number of Stories in each of the Release categories  4 Total Dependents  3  Dependent Entities  1 Dependently Attribute  Yes  No  Percentage  of User Stories Expected to be Depicted  30  Actual Depiction  20  Deposition  15  Defects  12  Bugs  11  Test Cases  10  User Story 1892536 As Is  1894783 As IS  1897297 As RRA  19010793 As AIAB  2201485 As RAID  20210797 As ALM B

In [96]:
story= "AI core | As an app I should be able to get the existing provisioning ENS notification once the app(RRA/RRA-plugin/RDB/RDB-plugin) is provisioned for a client/dc"

desc ='AI Core should Compute recommendations for the existing data of a client/dc when it is provisioned for RRA/RRA-Plugin/RDB/RDB-Plugin and merge them in the phoenix'


In [97]:
generate_acceptance_criteria(story,desc)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'AI core | As an app I should be able to get the existing provisioning ENS notification once the app(RRA/RRA-plugin/RDB/RDB-plugin) is provisioned for a client/dc\n\nAI Core should Compute recommendations for the existing data of a client/dc when it is provisioned for RRA/RRA-Plugin/RDB/RDB-Plugin and merge them in the phoenix\n\nAcceptance Criteria: App must have Acceptance criteria in place of Requirement List App can accept or reject the acceptance criteria   <SEP> App should receive the Approver recommendation from the AppServiceUId of the client and DC Upon provision to the application the approver will receive an automated email notification when the provision is completed and provision will be done in a timely manner  The App serviceUID will not be changed from Client to DC Once the approval has been provided by App ServiceUID App will automatically update to Client Admin User will have the option to manually update the APNTO DELETE option in settings  Apps need to ensure that a

In [ ]:
output_text = model.generate(input_ids=inputs,
                                 max_length=1024,
                                 temperature=0.7,
                                 do_sample=True,
                                 top_k=50,
                                 top_p=0.95,
                                 num_return_sequences=1)

    # Decode the generated output text
generated_text = tokenizer.decode(output_text[0], skip_special_tokens=True)

In [ ]:
import numpy as np
np.arange(10).shape

In [ ]:
np.arange(10)[:, np.newaxis